<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/before_framestacking/Auto_Combining_from_multiple_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [1]:
!pip install procgen

In [2]:
#Clone git
!git clone -b restructure_code https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

fatal: destination path 'my_project' already exists and is not an empty directory.


In [3]:
#update git
%cd /content/my_project
! git pull

/content/my_project
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project
   f70f3f1..4b2dab1  before_framestacking -> origin/before_framestacking
Already up to date.


In [4]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import

In [5]:
#Import all custom files
import baseline
import datatools as tools
import hyperparameters as h
import model
import my_util
import policy
import ppo
import utils

#other imports
import torch

#import specific methods
from baseline import set_hyperparameters
from ppo import PPO
from experiments import run_experiment, print_list_of_experiments


# Create Index File



In [14]:
INDEX = tools.create_index_table_from_txt_files()

# Create .csv's

Assumes that all file_names are correct! (with step count etc)

## Code

In [11]:
def create_training_csv_from_multiple_runs(base_file_name):
  all_related_files = find_related_csv_files(base_file_name)

  #create a df for each file
  dfs = []
  for name in all_related_files:
    dfs.append(pd.read_csv(DATA_PATH + name + ".csv"))

  dfs = remove_overlap_from_dfs(dfs)

  #Concatenate
  final_df = pd.concat(dfs)
  final_df = final_df.set_index("Step")

  #Save to file
  final_df.to_csv(DATA_PATH +base_file_name +"_combined.csv")

def create_EVAL_csv_from_multiple_runs(base_file_name):
  all_related_files = find_related_csv_files(base_file_name)
  base = 32*256*16

  dfs = []
  for name in all_related_files:
    df = pd.read_csv(DATA_PATH + name + "_EVAL.csv")
    #add steps if missing
    if "step" not in df.columns:
      df.insert(0, "Step", [base*(i+1) for i in range(len(df))])
    else:
      df.rename(columns={'step':'Step'}, inplace=True)
    dfs.append(df)

  dfs = remove_overlap_from_dfs(dfs)

  #Remove whitespace in headers
  for df in dfs:
    df.rename(columns=lambda x: x.strip(), inplace=True)

    #Concatenate
  final_df = pd.concat(dfs)
  final_df = final_df.set_index("Step")

  #save in file
  final_df.to_csv(DATA_PATH +base_file_name +"_combined_EVAL.csv")


def remove_overlap_from_dfs(dfs):
  #Dataframes overlap, so cut off excess
  breakpoints=[]
  for df in dfs[1:]:
    breakpoints.append(df['Step'][0])
  for i in range(len(dfs)-1):
    dfs[i] = dfs[i][dfs[i]["Step"] < breakpoints[i]]
  return dfs

def find_related_csv_files(base_file_name):
  file_names = [x.strip() for x in INDEX['file_name']]
  all_related_files= []
  for name in file_names:
    if base_file_name in name and "combined" not in name:
      all_related_files.append(name)

  return all_related_files


## Create files

In [12]:
base_file_name = "Experiment8_200levels_Run_06Dec_16h17m59s"
create_EVAL_csv_from_multiple_runs(base_file_name)
create_training_csv_from_multiple_runs(base_file_name)

In [13]:
base_file_name = "Experiment7_200levels_Run_06Dec_16h18m31s"
create_EVAL_csv_from_multiple_runs(base_file_name)
create_training_csv_from_multiple_runs(base_file_name)